<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 18.0 MB/s eta 0:00:00


In [95]:
import zipfile
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse
from sklearn.preprocessing import MinMaxScaler

import implicit
from datetime import datetime, timedelta


/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [78]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-78-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [79]:
len(df)

2220299

In [80]:
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)

In [81]:
df["Sku"] = df["Sku"].astype(str)

In [83]:
len(df[df["Qty Shipped"] == 0])

113860

In [84]:
# dropping retours
df = df[df["Qty Shipped"]>=0]

In [85]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [86]:
len(df)

737274

In [87]:
df.loc[:, "purchase"] = 1

In [88]:
df_1 = df[["BranchCustomerNbr","Sku","Qty Shipped", "purchase"]]

In [89]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737274 entries, 7402 to 2220294
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   BranchCustomerNbr  737274 non-null  object 
 1   Sku                737274 non-null  object 
 2   Qty Shipped        737274 non-null  float64
 3   purchase           737274 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 28.1+ MB


In [90]:
df_1

,BranchCustomerNbr,Sku,Qty Shipped,purchase
7402,44504136,CF58186,1.0,1
7902,44423496,406A863,34.0,1
7903,44538627,CC93753,3.0,1
8687,44880084,Y340805,11.0,1
9035,44443232,I440972,1.0,1
...,...,...,...,...
2220285,44424844,CD51044,1.0,1
2220286,44424844,2M365F5,4.0,1
2220290,44457581,V358351,2.0,1
2220291,44411461,CF48091,165.0,1


In [91]:
grouped_df = df.groupby(["BranchCustomerNbr", "Sku"]).agg({
    "Qty Shipped": "sum",
    "purchase": "sum"
}).reset_index()

In [92]:
grouped_df.sort_values(by="BranchCustomerNbr",ascending=False)

,BranchCustomerNbr,Sku,Qty Shipped,purchase
287317,44S312B2,CE91064,2.0,1
287316,44S309B2,CE35137,2.0,1
287315,44S304B2,CD69902,3.0,1
287314,44S201B2,CE35161,6.0,2
287313,44S201B2,091NR91,6.0,2
...,...,...,...,...
32,15413453,CE23704,1.0,1
31,15413453,CE17829,3.0,1
30,15413453,CE11999,2.0,1
29,15413453,CE10010,5.0,2


In [93]:
# Create a numeric user_id and artist_id column
grouped_df['BranchCustomerNbr'] = grouped_df['BranchCustomerNbr'].astype("category")
grouped_df['Sku'] = grouped_df['Sku'].astype("category")
grouped_df['bcn_id'] = grouped_df['BranchCustomerNbr'].cat.codes
grouped_df['sku_id'] = grouped_df['Sku'].cat.codes

In [94]:
grouped_df

,BranchCustomerNbr,Sku,Qty Shipped,purchase,bcn_id,sku_id
0,15413453,1413615,1.0,1,0,2285
1,15413453,1413616,4.0,2,0,2286
2,15413453,2228023,4.0,1,0,2810
3,15413453,270B716,2.0,1,0,4437
4,15413453,2M2GY64,2.0,1,0,5561
...,...,...,...,...,...,...
287313,44S201B2,091NR91,6.0,2,9889,1822
287314,44S201B2,CE35161,6.0,2,9889,28528
287315,44S304B2,CD69902,3.0,1,9890,26252
287316,44S309B2,CE35137,2.0,1,9891,28516


In [114]:
skus = grouped_df[["Sku","sku_id"]].drop_duplicates()
len(skus)

49487

In [119]:
users = grouped_df[["BranchCustomerNbr","bcn_id"]].drop_duplicates()
users[400:500]

,BranchCustomerNbr,bcn_id
13444,15886812,400
13445,15886964,401
13449,15886978,402
13464,15886987,403
13471,15887000,404
...,...,...
14389,44267800,495
14392,44292610,496
14393,44295070,497
14394,44297360,498


In [96]:
sparse_item_user = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['sku_id'], grouped_df['bcn_id'])))
sparse_user_item = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [109]:
sparse_item_user

<49487x9893 sparse matrix of type '<class 'numpy.float64'>'
	with 287318 stored elements in Compressed Sparse Row format>

**Als Model**

In [124]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

In [126]:
alpha_val = 40
data_conf = (sparse_user_item * alpha_val).astype('float')


model.fit(data_conf)

  0%|          | 0/20 [00:00<?, ?it/s]

In [131]:
# Assuming you want recommendations for user with ID 10
user_id = 899

# Get the row of the user from the sparse_user_item matrix
user_row = sparse_user_item[user_id]
# Reshape the user_row to have 1 row per user
user_items = np.reshape(user_row, (1, -1))

# Now you can call the recommend function
recommended = model.recommend(user_id, user_items, N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab
rec_tab

,sku_id,conf,Sku
0,2842.0,1.142565,2228626
1,24342.0,1.063562,CD07000
2,2908.0,1.038472,222A049
3,25310.0,1.027769,CD43050
4,17678.0,1.014608,CA26687
5,7763.0,1.010107,2U60071
6,3078.0,1.008682,222B914
7,43311.0,1.004869,N150920
8,22582.0,1.002924,CC18244
9,39398.0,1.002795,CG68127


In [128]:
#Get similar items
item_id = 7
n_similar = 3
similar = model.similar_items(item_id, n_similar)
print(similar)

(array([ 1553,     7, 45067], dtype=int32), array([1.0000001, 1.0000001, 1.       ], dtype=float32))
